In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import os, fnmatch
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures 
import seaborn as sns
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
%matplotlib inline

In [2]:
all_fits_files = []
listOfFiles = os.listdir('./')
pattern = "*.fits"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            all_fits_files.append(entry)

In [3]:
all_fits_files

['fbs1477_cor.fits']

In [4]:
obj_df = pd.read_csv("astro_objects.csv");
obj_df.head()

,Unnamed: 0,Unnamed: 0.1,plate,object_id,ra,dec,mag_B,mag_R,spectrum_class,spectrum_length,...,9,10,11,12,13,14,15,16,fits_path,jpeg_path
0,0,0,fbs0005,DFBSJ142114.64+425823.6,14:21:14.640,+42:58:23.63,13.86,13.15,OK,101,...,-2.189981e-11,-3.665937e-14,-9.373043e-12,-1.532294e-09,1.024176e-11,-2.280213e-14,2.109259e-17,-6.999758e-21,specimages/fbs0005-DFBSJ142114.64+425823.6.fits,specimagesjpeg/fbs0005-DFBSJ142114.64+425823.6...
1,1,1,fbs0005,DFBSJ142122.01+425921.3,14:21:22.012,+42:59:21.37,12.85,12.20,OK,106,...,-3.792762e-11,-3.647929e-14,-1.208479e-11,-1.865856e-09,1.801383e-11,-4.706942e-14,4.814870e-17,-1.724178e-20,specimages/fbs0005-DFBSJ142122.01+425921.3.fits,specimagesjpeg/fbs0005-DFBSJ142122.01+425921.3...
2,2,2,fbs0005,DFBSJ142125.68+425138.2,14:21:25.688,+42:51:38.29,16.38,14.85,NL,83,...,-1.815388e-11,-1.309881e-14,-3.936086e-12,-6.570811e-10,3.287456e-12,-6.007407e-15,4.796814e-18,-1.419620e-21,specimages/fbs0005-DFBSJ142125.68+425138.2.fits,specimagesjpeg/fbs0005-DFBSJ142125.68+425138.2...
3,3,3,fbs0005,DFBSJ142126.80+425301.4,14:21:26.802,+42:53:01.40,14.45,13.57,NL,120,...,-4.077909e-11,-2.942382e-14,-8.841611e-12,-1.475995e-09,8.681218e-12,-1.793400e-14,1.573814e-17,-5.009265e-21,specimages/fbs0005-DFBSJ142126.80+425301.4.fits,specimagesjpeg/fbs0005-DFBSJ142126.80+425301.4...
4,4,4,fbs0005,DFBSJ142141.43+430014.5,14:21:41.436,+43:00:14.59,16.39,16.47,NL,90,...,-1.024508e-12,-1.714980e-15,-4.384846e-13,-7.168274e-11,5.490072e-13,-1.330955e-15,1.317058e-18,-4.632790e-22,specimages/fbs0005-DFBSJ142141.43+430014.5.fits,specimagesjpeg/fbs0005-DFBSJ142141.43+430014.5...


In [5]:
plate_id = all_fits_files[0].split("_")[0]
print(plate_id)

fbs1477


In [6]:
this_plate_df = obj_df.loc[obj_df['plate'] == plate_id] 

In [7]:
this_plate_df

,Unnamed: 0,Unnamed: 0.1,plate,object_id,ra,dec,mag_B,mag_R,spectrum_class,spectrum_length,...,9,10,11,12,13,14,15,16,fits_path,jpeg_path
746,746,746,fbs1477,DFBSJ081755.98+190314.9,08:17:55.984,+19:03:14.99,15.87,13.68,U,108,...,-4.146239e-11,-6.940651e-14,-1.774584e-11,-2.901079e-09,1.490176e-11,-2.834247e-14,2.360833e-17,-7.270340e-21,specimages/fbs1477-DFBSJ081755.98+190314.9.fits,specimagesjpeg/fbs1477-DFBSJ081755.98+190314.9...
747,747,747,fbs1477,DFBSJ081756.25+185537.2,08:17:56.255,+18:55:37.25,15.19,14.40,OK,109,...,3.351549e-12,2.418313e-15,7.266861e-13,1.213128e-10,4.547560e-15,-8.282495e-16,1.165559e-18,-4.622449e-22,specimages/fbs1477-DFBSJ081756.25+185537.2.fits,specimagesjpeg/fbs1477-DFBSJ081756.25+185537.2...
748,748,748,fbs1477,DFBSJ081757.83+185724.2,08:17:57.830,+18:57:24.24,16.22,15.27,NL,120,...,-6.516964e-12,-4.702264e-15,-1.412991e-12,-2.358810e-10,1.351952e-12,-2.763035e-15,2.417256e-18,-7.701931e-22,specimages/fbs1477-DFBSJ081757.83+185724.2.fits,specimagesjpeg/fbs1477-DFBSJ081757.83+185724.2...
749,749,749,fbs1477,DFBSJ081758.22+185818.0,08:17:58.226,+18:58:18.03,15.88,15.10,NL,85,...,4.439892e-12,3.203587e-15,9.626533e-13,1.607039e-10,-4.890589e-13,5.019760e-16,-1.761059e-19,3.065624e-24,specimages/fbs1477-DFBSJ081758.22+185818.0.fits,specimagesjpeg/fbs1477-DFBSJ081758.22+185818.0...
750,750,750,fbs1477,DFBSJ081759.08+190328.7,08:17:59.083,+19:03:28.76,15.22,13.93,OK,78,...,-2.153562e-11,-1.553885e-14,-4.669294e-12,-7.794794e-10,4.595515e-12,-9.488847e-15,8.317449e-18,-2.644268e-21,specimages/fbs1477-DFBSJ081759.08+190328.7.fits,specimagesjpeg/fbs1477-DFBSJ081759.08+190328.7...
751,751,751,fbs1477,DFBSJ081759.45+185609.4,08:17:59.455,+18:56:09.46,14.22,13.46,OK,92,...,1.678281e-11,1.210959e-14,3.638842e-12,6.074639e-10,-1.365780e-12,3.017734e-16,1.087211e-18,-6.276496e-22,specimages/fbs1477-DFBSJ081759.45+185609.4.fits,specimagesjpeg/fbs1477-DFBSJ081759.45+185609.4...
752,752,752,fbs1477,DFBSJ081800.86+185521.1,08:18:00.860,+18:55:21.16,16.12,14.98,U,75,...,-1.670799e-12,-1.205542e-15,-3.622541e-13,-6.047317e-11,5.827585e-13,-1.439466e-15,1.372935e-18,-4.554358e-22,specimages/fbs1477-DFBSJ081800.86+185521.1.fits,specimagesjpeg/fbs1477-DFBSJ081800.86+185521.1...
753,753,753,fbs1477,DFBSJ081803.04+185629.5,08:18:03.049,+18:56:29.53,15.43,14.80,OK,82,...,4.047396e-12,6.775215e-15,1.732288e-12,2.831947e-10,-9.472519e-13,1.126492e-15,-5.336678e-19,7.215335e-23,specimages/fbs1477-DFBSJ081803.04+185629.5.fits,specimagesjpeg/fbs1477-DFBSJ081803.04+185629.5...
754,754,754,fbs1477,DFBSJ081804.21+190235.0,08:18:04.218,+19:02:35.09,15.08,13.71,OK,120,...,-2.187956e-11,-3.662556e-14,-9.364409e-12,-1.530887e-09,8.384043e-12,-1.662733e-14,1.424726e-17,-4.474554e-21,specimages/fbs1477-DFBSJ081804.21+190235.0.fits,specimagesjpeg/fbs1477-DFBSJ081804.21+190235.0...
755,755,755,fbs1477,DFBSJ081804.60+190012.6,08:18:04.602,+19:00:12.64,16.81,15.99,U,90,...,1.031904e-13,-2.887247e-15,-7.674141e-13,-1.229793e-10,7.037717e-13,-1.446200e-15,1.277935e-18,-4.127269e-22,specimages/fbs1477-DFBSJ081804.60+190012.6.fits,specimagesjpeg/fbs1477-DFBSJ081804.60+190012.6...


In [8]:
fits_files = WCS(all_fits_files[0]) 

In [9]:
obj_loc_id = 746
obj_ra = this_plate_df.loc[obj_loc_id]["ra"]
obj_dec = this_plate_df.loc[obj_loc_id]["dec"] 
c = SkyCoord(obj_ra, obj_dec, unit=(u.hourangle, u.deg))
c


<SkyCoord (ICRS): (ra, dec) in deg
    (124.48326667, 19.05416389)>

In [10]:
dx, dy = fits_files.all_world2pix(c.ra, c.dec, 0)

In [11]:
print(dx)
print(dy)

4928.6026005358735
4893.714762954662
